In [19]:
import pandas as pd
import os
import psycopg2 as pg
from psycopg2 import sql
import plotly.express as px

In [20]:
valid_answer = True
while (valid_answer):
    answer = input('Use Cloud DB? (y/n):').lower()
    if answer.lower() == 'n' or answer.lower() == 'no':
        db_host = os.environ.get('DB_HOST')
        db_port = os.environ.get('DB_PORT')
        db_user = os.environ.get('DB_USER')
        db_password = os.environ.get('DB_PASSWORD')
        db_name = os.environ.get('DB_NAME')
        valid_answer = False
        print('Using local DB')
    elif answer.lower() == 'y' or answer.lower() =='yes':
        db_host = os.environ.get('DB_HOST')
        db_port = os.environ.get('DB_PORT')
        db_user = os.environ.get('DB_USER')
        db_password = os.environ.get('DB_PASSWORD')
        db_name = os.environ.get('DB_NAME')
        valid_answer = False
        print('Using Cloud DB')
    else:
        print('Invalid input. Please enter y or n.')

db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

Using local DB


In [21]:
# read country_areas csv into pandas dataframe
cdf = pd.read_csv('data/country_areas.csv', index_col=0).sort_values(by='name')
cdf.drop(columns=['reported_area'], inplace=True)
cdf.rename(columns={'calculated_area': 'area'}, inplace=True)
# read as whole numbers instead of exponential
# cdf['area'] = cdf['area'].apply(lambda x: int(x))
cdf.head()

,area
name,
Abyei,9.978891e+03
Afghanistan,6.420495e+05
Aksai Chin,3.066755e+04
Albania,2.864390e+04
Algeria,2.312184e+06


In [22]:
# connect to the database
conn = pg.connect(
    database=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

In [23]:
# read country_areas table into pandas dataframe
sql = '''
SELECT
    country_name,
    SUM(
    transition_0 + 
    + transition_1 
    + transition_2 
    + transition_3
    + transition_4
    + transition_5
    + transition_6
    + transition_7
    + transition_8
    + transition_9
    + transition_10
    ) / 1000000 
    AS calculated_area_km2_with_unassigned_land
    FROM cep_water
    WHERE is_marine = FALSE
    GROUP BY country_name
    ORDER BY country_name
'''
uacepdf = pd.read_sql_query(sql, conn).set_index('country_name')
uacepdf.head()

C:\Users\riyad\AppData\Local\Temp\ipykernel_23212\2755111723.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  uacepdf = pd.read_sql_query(sql, conn).set_index('country_name')


,calculated_area_km2_with_unassigned_land
country_name,
Afghanistan,6.418377e+05
Albania,2.869214e+04
Algeria,2.308858e+06
American Samoa,2.092911e+02
Andorra,4.746219e+02


In [24]:
# read country_areas table into pandas dataframe
sql = '''
SELECT
    country_name,
    SUM(
    transition_0 + 
    + transition_1 
    + transition_2 
    + transition_3
    + transition_4
    + transition_5
    + transition_6
    + transition_7
    + transition_8
    + transition_9
    + transition_10
    ) / 1000000 
    AS calculated_area_km2
    FROM cep_water
    WHERE is_marine = FALSE
    AND eco != 100001
    GROUP BY country_name
    ORDER BY country_name
'''
lcepdf = pd.read_sql_query(sql, conn).set_index('country_name')
lcepdf.head()

C:\Users\riyad\AppData\Local\Temp\ipykernel_23212\832246961.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lcepdf = pd.read_sql_query(sql, conn).set_index('country_name')


,calculated_area_km2
country_name,
Afghanistan,6.418377e+05
Albania,2.842427e+04
Algeria,2.308794e+06
American Samoa,1.313471e+02
Andorra,4.746219e+02


In [25]:
df = uacepdf.join(lcepdf, how='inner')
df.head()

,calculated_area_km2_with_unassigned_land,calculated_area_km2
country_name,,
Afghanistan,6.418377e+05,6.418377e+05
Albania,2.869214e+04,2.842427e+04
Algeria,2.308858e+06,2.308794e+06
American Samoa,2.092911e+02,1.313471e+02
Andorra,4.746219e+02,4.746219e+02


In [26]:
# merge the two dataframes and calculate the percentage difference between the two area columns
df = cdf.merge(df, left_index=True, right_index=True, how='inner')
df['diff_unassigned'] = ( df['calculated_area_km2_with_unassigned_land']- df['area'] ) / df['area'] * 100
df['diff'] = ( df['calculated_area_km2']- df['area']) / df['area'] * 100
# rename columns
df.columns = ['GEE_area', 'CEP_area_unassigned','CEP_area' ,'diff_unassigned_percent','diff_percent']
df.head()

,GEE_area,CEP_area_unassigned,CEP_area,diff_unassigned_percent,diff_percent
Afghanistan,6.420495e+05,6.418377e+05,6.418377e+05,-0.032982,-0.032982
Albania,2.864390e+04,2.869214e+04,2.842427e+04,0.168440,-0.766746
Algeria,2.312184e+06,2.308858e+06,2.308794e+06,-0.143845,-0.146586
American Samoa,2.007186e+02,2.092911e+02,1.313471e+02,4.270926,-34.561587
Andorra,4.738402e+02,4.746219e+02,4.746219e+02,0.164972,0.164972


In [27]:
# get row where max difference
df.loc[df['diff_percent'].idxmax()]

GEE_area                    8.417049
CEP_area_unassigned        10.563452
CEP_area                   10.563452
diff_unassigned_percent    25.500671
diff_percent               25.500671
Name: Monaco, dtype: float64

In [28]:
# get slovenia row
df.loc['Slovenia']

GEE_area                   20269.978613
CEP_area_unassigned        20327.427286
CEP_area                   20313.114365
diff_unassigned_percent        0.283418
diff_percent                   0.212806
Name: Slovenia, dtype: float64

In [29]:
df.loc[df['diff_percent'].idxmin()]

GEE_area                   224.203857
CEP_area_unassigned        301.889311
CEP_area                    14.389841
diff_unassigned_percent     34.649473
diff_percent               -93.581805
Name: Maldives, dtype: float64

In [30]:
# get rows where difference is greater than 50% or less than -50%
df[(df['diff_percent'] > 50) | (df['diff_percent'] < -50)]

,GEE_area,CEP_area_unassigned,CEP_area,diff_unassigned_percent,diff_percent
Bermuda,63.236821,72.255170,27.115404,14.261230,-57.120862
British Indian Ocean Territory,49.706153,26.456033,16.626793,-46.775135,-66.549829
Christmas Island,136.341080,141.520392,36.158384,3.798791,-73.479465
Cocos (Keeling) Islands,15.832597,17.806543,3.973882,12.467610,-74.900633
Kiribati,1024.711471,1025.835391,178.625771,0.109682,-82.568189
Maldives,224.203857,301.889311,14.389841,34.649473,-93.581805
Marshall Islands,269.104485,277.985100,43.183604,3.300062,-83.952849
Seychelles,501.426576,483.185715,207.727583,-3.637793,-58.572682
Tuvalu,47.722799,41.387317,5.204732,-13.275587,-89.093825


In [31]:
# total area difference
a = df['GEE_area'].sum()
b = df['CEP_area'].sum()
print(f'Total area difference: {((a - b) / ((a+b)/2) * 100).round(2)}%')

Total area difference: 0.31%


In [32]:
print(len(df), len(cdf), len(uacepdf), len(lcepdf))

216 276 276 265


In [33]:
# check which countries are missing
missing = uacepdf[~uacepdf.index.isin(cdf.index)]
missing

,calculated_area_km2_with_unassigned_land
country_name,
Argentina|Falkland Islands (Malvinas),5.690371e+02
Argentina|South Georgia and the South Sandwich Islands,2.711528e+02
Argentina|Uruguay,1.917254e-01
Australia|Papua New Guinea,2.088737e+01
Bolivia (Plurinational State of),1.084467e+06
"Bonaire, Sint Eustatius and Saba",3.209840e+02
British Indian Ocean Territory|Mauritius,4.196682e+01
Brunei Darussalam|China|Malaysia|Philippines|Viet Nam,3.970889e-01
Cabo Verde,4.125213e+03


In [36]:
# see algeria in normal numbers instead of exponential
print(df.loc['Turkmenistan'])

GEE_area                   554569.004490
CEP_area_unassigned        555052.515853
CEP_area                   555052.515853
diff_unassigned_percent         0.087187
diff_percent                    0.087187
Name: Turkmenistan, dtype: float64


In [ ]:
# save to csv
df.to_csv('data/area_diff.csv')

In [ ]:
conn.close()